# Import Libraries

In [2]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

#models
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.tree import DecisionTreeRegressor

# Metrics

In [3]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

def MAPE(y_test, pred):
    mape = np.mean(np.abs((y_test - pred) / y_test))
    return mape

# Parameters

In [4]:
label_name = "y"
test_size = 0.3
date_column_name = "date"
model_name = "lightgbm"

# Read Data

In [5]:
data_path = "DailyDelhiClimate.csv"

In [6]:
df = pd.read_csv(data_path)
df = df.drop("Unnamed: 0",axis=1)

if date_column_name is not None:
    df = df.drop(date_column_name, axis=1)
col_list = list(df.columns)
col_list.remove(label_name)
col_list.insert(0, label_name)

df = df[col_list]

y = df.loc[:,"y"]
X = df.iloc[:,1:]

data_len = len(X)
forecast_horizion = int(data_len * test_size)

X_train, X_test = X.iloc[:-forecast_horizion], X.iloc[-forecast_horizion:]
y_train, y_test = y.iloc[:-forecast_horizion], y.iloc[-forecast_horizion:]

# Model

In [11]:
model_dic = {
    "lightgbm": lgb.LGBMRegressor(),
    "xgboost": xgb.XGBRegressor(),
    "catboost": cb.CatBoostRegressor(),
    "decision_tree": DecisionTreeRegressor()
 }

search_param_dic = {
    "lightgbm": {
            "n_estimators": [100, 250, 500, 750, 1000],
            "learning_rate": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
            "num_leaves": [15, 31, 63, 127, 255],
            "max_depth": [4, 6, 7, 8, 10],
            "subsample": [0.4, 0.6, 0.7, 0.9],
            "subsample_freq": [1, 5, 10, 20, 50],
            "colsample_bytree": [0.4, 0.6, 0.7, 0.9],
            "reg_alpha": [0, 0.01, 0.05, 0.5, 1, 10],
            "reg_lambda": [0, 0.01, 0.05, 0.5, 1, 10],
            "max_bin": [15, 31, 63, 127, 255],
            "random_state": [0],
            "verbose": [-1],
        },
    "xgboost": {
            "n_estimators": [100, 250, 500, 750, 1000],
            "learning_rate": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
            "num_leaves": [15, 31, 63, 127, 255],
            "max_depth": [4, 6, 7, 8, 10],
            "subsample": [0.4, 0.6, 0.7, 0.9],
            "subsample_freq": [1, 5, 10, 20, 50],
            "colsample_bytree": [0.4, 0.6, 0.7, 0.9],
            "reg_alpha": [0, 0.01, 0.05, 0.5, 1, 10],
            "reg_lambda": [0, 0.01, 0.05, 0.5, 1, 10],
            "max_bin": [15, 31, 63, 127, 255],
            "random_state": [0],
            "verbose": [-1],
        },
    "catboost": {
        "iterations": [100, 300, 500, 1000],
        "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
        "depth": [4, 6, 8, 10],
        "l2_leaf_reg": [1, 3, 5, 7, 9],
        "border_count": [32, 64, 128, 254],
        "random_strength": [0, 0.1, 0.5, 1],
        "random_seed": [0],
    }
,
    "decision_tree": {
        "criterion": ["gini", "entropy"],
        "max_depth": [2, 3, 4, 5, 10, 12],
        "min_samples_split": [2, 5, 10, 20],
        "min_samples_leaf": [1, 2, 4, 10],
}
 }


model = model_dic[model_name]
searching_params = search_param_dic[model_name]

# Parameter Search

In [12]:
# Hold-out Validation
xv_cls = RandomizedSearchCV
val_size = forecast_horizion
train_val_indexes = np.zeros_like(y_train)
train_val_indexes[:-val_size] = -1
fold_size = PredefinedSplit(test_fold=train_val_indexes)

xv = xv_cls(estimator=model, param_distributions=searching_params, n_iter=10000, scoring="neg_mean_absolute_error", n_jobs=-1,
                    cv=fold_size, verbose=-1, refit=False)

xv.fit(X_train, y_train)

best_params = xv.best_params_

# Fit & Predict

In [ ]:
X_train, X_test = X.iloc[:-forecast_horizion], X.iloc[-forecast_horizion:]
y_train, y_test = y.iloc[:-forecast_horizion], y.iloc[-forecast_horizion:]

best_model = type(model)(**best_params).fit(X_train, y_train)
preds = best_model.predict(X_train)
fores = best_model.predict(X_test)

# Score Calculation

In [ ]:
train_mse_score = mse(y_train,preds)
train_mae_score = mae(y_train,preds)
train_mape_score = MAPE(y_train,preds)

test_mse_score = mse(y_test,fores)
test_mae_score = mae(y_test,fores)
test_mape_score = MAPE(y_test,fores)

# PLOT

In [ ]:
plt.plot(y_train,label = "target")
plt.plot(preds,label="preds")
plt.legend()
plt.title("Train Targets vs Train Preds")
plt.show()

plt.plot(y_test,label = "target")
plt.plot(fores,label="fores")
plt.legend()
plt.title("Test Labels vs Test Forecasts")
plt.show()

#######################################

# SARIMAX

# LSTM

In [14]:

df = pd.read_csv(data_path)
df = df.drop("Unnamed: 0",axis=1)

if date_column_name is not None:
    df = df.drop(date_column_name, axis=1)
col_list = list(df.columns)
col_list.remove(label_name)
col_list.insert(0, label_name)

df = df[col_list]

y = df.loc[:,"y"]
X = df.iloc[:,1:]

y = y.values
X = X.values

data_len = len(X)
forecast_horizion = int(data_len * test_size)

y_train_orig = y[:-forecast_horizion]
y_test_orig = y[-forecast_horizion:]

feature_shape = X.shape[1]

In [15]:

# Normalize the data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Create sequences for input and output
def create_sequences(data, target, n_steps):
    X_seq, y_seq = [], []
    for i in range(len(data) - n_steps):
        X_seq.append(data[i : i + n_steps])
        y_seq.append(target[i + n_steps])
    return np.array(X_seq), np.array(y_seq)

n_steps = 10  # number of time steps to look back
X_seq, y_seq = create_sequences(X_scaled, y_scaled, n_steps)

# Split the data into training and testing sets

X_train, X_test = X_seq[:-forecast_horizion], X_seq[-forecast_horizion:]
y_train, y_test = y_seq[:-forecast_horizion], y_seq[-forecast_horizion:]


In [17]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, activation="relu", input_shape=(n_steps, feature_shape)))
model.add(Dense(units=1))
model.compile(optimizer="adam", loss="mean_absolute_error")

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)

print(f"Training Loss: {train_loss}")
print(f"Test Loss: {test_loss}")

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)



Epoch 1/100

34/34 [==============================] - 1s 4ms/step - loss: 0.1209
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0698
Epoch 3/100
34/34 [==============================] - 0s 5ms/step - loss: 0.0608
Epoch 4/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0570
Epoch 5/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0580
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0554
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0545
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0521
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0529
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0509
Epoch 11/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0497
Epoch 12/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0500
Epoch 13/100
34/34 [==============

In [16]:
# Inverse transform the predictions to the original scale
y_train_pred_inv = scaler_y.inverse_transform(y_train_pred).reshape(-1)
y_test_pred_inv = scaler_y.inverse_transform(y_test_pred).reshape(-1)
y_train = y_train_orig.copy()
y_test = y_test_orig.copy()

#Calculate Score
train_mse_score = mse(y_train, y_train_pred_inv)
train_mae_score = mae(y_train, y_train_pred_inv)
train_mape_score = MAPE(y_train, y_train_pred_inv)

test_mse_score = mse(y_test, y_test_pred_inv)
test_mae_score = mae(y_test, y_test_pred_inv)
test_mape_score = MAPE(y_test, y_test_pred_inv)

NameError: name 'y_train_pred' is not defined